# Lines spectral clustering

This notebook focuses on the separation of lines into several sets sharing similar properties. The objectives are diverse:
- It is possible to physically interpret the clusters thus created, which will be carried out in this notebook.
- It is possible to use this decomposition to train several specialized neural networks. This is described in `nn-regression-fc-clustering.ipynb` and `nn-regression-dense-clustering.ipynb`.

To achieve this, we perform a [spectral clustering](https://en.wikipedia.org/wiki/Spectral_clustering) on the correlation matrix (Pearson or Spearman) of the lines. To do so, we use the [scikit-learn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html).

In [ ]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(""), ".."))

import shutil
import json
import itertools as itt
from math import pi, ceil

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Rectangle

from scipy.stats import spearmanr as _spearmanr
from sklearn.cluster import SpectralClustering
from sklearn.decomposition import PCA

from helpers.lines import (
    molecules_among_lines,
    filter_molecules,
    molecule_and_transition,
    molecule_to_latex,
)
from helpers.preprocessing import prepare_data

plt.rc("text", usetex=shutil.which("latex") is not None)

dirname = os.path.join(os.path.splitext(os.path.abspath(""))[0], "out-clustering")

## Data loading

In [ ]:
train_set, val_set, train_mask_set, val_mask_set = prepare_data()

_, df_train = train_set.to_pandas()  # Already in log scale
_, df_val = train_set.to_pandas()  # Already in log scale
df_mask_train = train_mask_set.to_pandas()
df_mask_val = val_mask_set.to_pandas()

lines_names = df_train.columns.to_list()

### Selection of a subset of lines

You can filter the lines you want to cluster here. By default, all lines are taken into account.

In [ ]:
lines_names = filter_molecules(lines_names, None)

df_train = df_train[lines_names]
df_val = df_val[lines_names]

In [ ]:
X_train = df_train.values
X_val = df_val.values

m_train = df_mask_train.values
m_val = df_mask_val.values

all_molecules = molecules_among_lines(lines_names)
all_molecules_number = [
    len(filter_molecules(lines_names, mol)) for mol in all_molecules
]

pd.DataFrame(
    data={"Molecules": all_molecules, "Lines": all_molecules_number}
).set_index(["Molecules", "Lines"])

## Pearson/Spearman correlation matrix

Spectral clustering needs a measure of similarity between features. By default, we choose linear correlation (Pearson coefficient), but it is possible to change to Spearman correlation, which takes into account monotonic non-linear relationships.

In [ ]:
def pearsonr(X, m=None):
    """m must be False when the sample must be ignored, else True"""
    if m is None:
        n_samples = X.shape[0]
        r = (
            1
            / n_samples
            * (X - X.mean(axis=0, keepdims=True)).T
            @ (X - X.mean(axis=0, keepdims=True))
        )
    else:
        X = np.ma.array(X, mask=~m.astype(bool))
        n_samples = m.sum(axis=0)
        r = (
            1
            / n_samples
            * (X - X.mean(axis=0, keepdims=True)).T.dot(
                X - X.mean(axis=0, keepdims=True)
            )
        )
    r /= X.std(axis=0).reshape(-1, 1) * X.std(axis=0).reshape(1, -1)
    return r


def spearmanr(X, m=None):
    if m is None:
        return _spearmanr(
            np.where(m_train, X_train, np.nan), nan_policy="omit"
        ).correlation
    return _spearmanr(X_train).correlation

In [ ]:
# Compute the correlation matrix
corr = pearsonr(X_train, m_train)  # Can be replaced by: spearmanr(X_train, m_train)

# Absolute value
corr = np.abs(corr)

# Ensure that corr is between 0 and 1
corr /= corr.max()

# Ensure that the matrix is perfectly symetric
corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1.0)

### Correlation matrix

Below, we display the correlation matrix (in absolute values) of the lines. We also show the most represented molecules (labels are not exhaustive for display purposes).

In [ ]:
# Lines ticks

middles = []
for mol in all_molecules:
    _ls = [molecule_and_transition(name)[0] for name in lines_names]
    start = _ls.index(mol)
    end = len(_ls) - 1 - _ls[::-1].index(mol)
    middles.append((start + end) / 2)

# Display the correlation matrix

plt.figure(dpi=125)

plt.imshow(corr, cmap="jet", vmin=0, vmax=1)
cbar = plt.colorbar()
cbar.set_label("Absolute Pearson coefficient", labelpad=15, rotation=270)

_ticks = [mid for mid, num in zip(middles, all_molecules_number) if num > 50]
_labels = [
    molecule_to_latex(mol)
    for mol, num in zip(all_molecules, all_molecules_number)
    if num > 50
]
plt.xticks(_ticks, labels=_labels, rotation=90, fontsize=8)
plt.yticks(_ticks, labels=_labels, rotation=0, fontsize=8)

plt.show()

## Spectral clustering

The spectral clustering will perform a segmentation into clusters with a high degree of similarity. A priori, there is no particular reason for the clusters to be balanced.

The number of clusters `n_clusters` can be modified.

In [ ]:
n_clusters = 4

spectral = SpectralClustering(
    n_clusters=n_clusters, affinity="precomputed", random_state=0, n_init=50
).fit(corr)
labels = spectral.labels_

clusters_indices = {
    k + 1: np.arange(labels.size)[labels == k].tolist() for k in range(n_clusters)
}

In [ ]:
clusters_lines = {
    k: [lines_names[i] for i in clusters_indices[k]] for k in clusters_indices
}

clusters_molecules = {
    k: [molecule_and_transition(line)[0] for line in clusters_lines[k]]
    for k in clusters_lines
}
clusters_transitions = {
    k: [molecule_and_transition(line)[1] for line in clusters_lines[k]]
    for k in clusters_lines
}

pd.DataFrame(
    {
        "Cluster": list(range(1, n_clusters + 1)),
        "Number of lines": [len(clusters_lines[k + 1]) for k in range(n_clusters)],
    }
).set_index(["Cluster", "Number of lines"]).transpose()

In [ ]:
clusters_ = clusters_lines

df_clusters = pd.DataFrame(
    columns=[f"cluster {i}" for i in range(1, 5)] + ["total"],
    index=all_molecules,
)
df_clusters["total"] = all_molecules_number

for i in range(4):
    all_molecules_cluster = molecules_among_lines(clusters_[i + 1])
    all_molecules_number_cluster = [
        len(filter_molecules(clusters_[i + 1], mol)) for mol in all_molecules_cluster
    ]
    df_clusters.loc[
        all_molecules_cluster, f"cluster {i+1}"
    ] = all_molecules_number_cluster


df_clusters = df_clusters.fillna(0)
df_clusters

We save the clusters to be used in `nn-regression-fc-clustering.ipynb` and `nn-regression-dense-clustering.ipynb`.

In [ ]:
filename = os.path.join(dirname, f"{n_clusters}_clusters.json")

if not os.path.isdir(dirname):
    os.mkdir(dirname)

with open(filename, "w", encoding="utf-8") as f:
    json.dump(clusters_lines, f, ensure_ascii=False, indent=4)

## Lines reordering

We reorder the lines in order to visualize the intra and inter-cluster correlations. As a matter of fact, lines of the same molecules are not grouped anymore so we cannot display the chemical species names.

In [ ]:
reordering = []
for k in clusters_indices:
    reordering += clusters_indices[k]

plt.figure(dpi=125)

plt.imshow(corr[reordering, :][:, reordering], vmin=0, vmax=1, cmap="jet")

plt.colorbar()

plt.show()

We draw (white line) the border of the clusters.

In [ ]:
plt.figure(dpi=125)

plt.imshow(corr[reordering, :][:, reordering], vmin=0, vmax=1, cmap="jet")

start = 0
for k in clusters_indices:

    width = len(clusters_indices[k])

    plt.gca().add_patch(
        Rectangle(
            (start, start), width, width, edgecolor="white", facecolor="none", lw=3
        )
    )

    start += width

plt.colorbar()

plt.show()

## Clusters analysis

In this section, we propose to analyse the composition of the clusters.

In [ ]:
# Molecular composition of clusters


def count_molecules(molecules, ratio=False):
    scores = [None] * len(all_molecules)
    for i, (molecule, number) in enumerate(zip(all_molecules, all_molecules_number)):
        if ratio:
            scores[i] = (
                100 * len([mol for mol in molecules if mol == molecule]) / number
            )
        else:
            scores[i] = len([mol for mol in molecules if mol == molecule])
    return all_molecules, scores


ratio = True

The following figure shows the repartition of each molecule within the different clusters.

In [ ]:
plt.figure(figsize=(1.5 * 6.4, 4.8), dpi=125)

offset = None
for k in clusters_lines:

    x, y = count_molecules(clusters_molecules[k], ratio=ratio)

    if offset is None:
        offset = [0 for _ in y]
    # [y + off for y, off in zip(y, offset)]
    plt.bar(
        list(range(len(x))),
        y,
        tick_label=[molecule_to_latex(xi) for xi in x],
        bottom=offset,
        alpha=0.8,
    )
    offset = [off + y for off, y in zip(offset, y)]

plt.xticks(rotation=60, fontsize=12)
plt.yticks(fontsize=12)
if ratio:
    plt.ylabel(
        "Proportion of representants of molecules (\%)", fontsize=14, labelpad=10
    )
else:
    plt.ylabel("Number of representants in cluster", fontsize=14)
# plt.title(f'Clusters ({len(lines_names)} lines)')

plt.show()

The following plots shows the fraction of each molecule that compose the different clusters. A red cross indicates that no line of this molecule is part of the cluster.

In [ ]:
for k in clusters_lines:

    x, y = count_molecules(clusters_molecules[k], ratio=ratio)

    plt.figure(figsize=(1.5 * 6.4, 0.6 * 4.8), dpi=125)

    plt.bar(
        list(range(len(x))),
        y,
        tick_label=[
            molecule_to_latex(xi) + f"({tot})"
            for xi, tot in zip(x, all_molecules_number)
        ],
        color="tab:blue",
    )
    if ratio:
        plt.axhline(100, color="red", linestyle="--")

    for i, _y in enumerate(y):
        if _y == 0:
            plt.plot(i, _y, marker="x", color="red", markersize=12)

    plt.xticks(rotation=60)
    if ratio:
        plt.ylabel("Proportion of molecules (\%)")
    else:
        plt.ylabel("Number of representants in cluster")
    # plt.title(f'Cluster n°{k} ({len(clusters_lines[k])} lines)')

    plt.show()

In [ ]:
def print_most_representative_lines(cluster_idx: int, corr, n_lines: int):
    corr_mean = corr.mean(axis=0)

    idx_max = np.argsort(corr_mean)[::-1]
    print(np.array(clusters_[f"{cluster_idx}"])[idx_max[:n_lines]])
    return corr_mean

In [ ]:
train_set, val_set, train_mask_set, val_mask_set = prepare_data()

theta_train, df_train = train_set.to_pandas()  # Already in log scale
_, df_val = train_set.to_pandas()  # Already in log scale
df_mask_train = train_mask_set.to_pandas()
df_mask_val = val_mask_set.to_pandas()

lines_names = df_train.columns.to_list()

theta = theta_train.values

In [ ]:
corr_mean = corr.mean(axis=0)

In [ ]:
def analysis_corr_with_inputs(corr_mean, X_cluster):
    idx_max = np.argmax(corr_mean)
    full_1 = np.column_stack((theta, X_cluster[:, idx_max]))

    corr_output = pearsonr(full_1)[:-1, -1]

    return corr_output

In [ ]:
# df_corr_output = pd.DataFrame(
#     columns=["line", "Pth", "G0", "AV"],
#     index=[f"cluster_{i+1}" for i in range(3)]
# )

# for cluster_idx in range(1, n_clusters+1):

#     X_cluster = df_train[clusters_[cluster_idx]].values
#     corr_output = analysis_corr_with_inputs(corr_mean, X_cluster)

#     idx_max = np.argmax(corr_mean)
#     df_corr_output.at[
#         f"cluster_{cluster_idx}",
#         "line"
#     ] = clusters_[cluster_idx][idx_max]
#     df_corr_output.loc[f"cluster_{cluster_idx}", ["Pth", "G0", "AV"]] = corr_output[:-1]

In [ ]:
# # ------- PART 1: Create background

# # number of variable
# categories = [r"$A_V^{tot}$", r"$G_0$", r"$P_{th}$"] #list(df_corr_output)[1:]
# N = len(categories)

# # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
# angles = [n / float(N) * 2 * pi for n in range(N)]
# angles += angles[:1]
# angles = np.array(angles)

# list_mols_ticks = [
#     molecule_to_latex(xi) + f" ({tot})"\
#     for xi, tot in zip(df_clusters.index, df_clusters["total"])
# ]
# list_mols_ticks.reverse()

# dict_order_clusters = {
#     1:1,
#     2:3,
#     3:4,
#     4:2
# }

# fig, ax = plt.subplots(
#     nrows=2,
#     ncols=4,
#     sharey='row',
#     dpi=150,
#     gridspec_kw={'height_ratios': [7.5, 1]},
#     figsize=(6,8)
# )

# fig.text(
#     0.6,
#     1.0,
#     'Proportion of lines from a species in a cluster',
#     size=14,
#     horizontalalignment='center',
# )

# fig.text(
#     0.6,
#     -0.02,
#     "Pearson correlations with main physical parameters",
#     size=14,
#     horizontalalignment='center',
# )


# N_mol = len(df_clusters)

# for cluster_idx,list_lines_cluster in clusters_.items():
#     i = dict_order_clusters[int(cluster_idx)] - 1

#     ax[0,i].barh(
#         np.arange(N_mol),
#         (100 * df_clusters[f"cluster_{cluster_idx}"] / df_clusters["total"]).values[::-1],
#         color='tab:blue',
#     )

#     for j, _y in enumerate(df_clusters[f"cluster_{cluster_idx}"].values[::-1]):
#         if _y == 0:
#             ax[0,i].plot(_y+10, j, marker='x', color='red', markersize=8)

#     ax[0,i].set_xticks([0, 50, 100])
#     ax[0,i].set_xticklabels([0, 0.5, 1], fontsize=12)
#     ax[0,i].set_xlim([0, 100])
#     ax[0,i].xaxis.tick_top()

#     ax[0,i].set_yticks(range(N_mol))
#     ax[0,i].set_yticklabels(list_mols_ticks, fontsize=12)
#     ax[0,i].set_ylim([-2/3, N_mol-1/3])

#     ax[0,i].tick_params(pad=0.05)

#     for k in range(1, N_mol, 5):
#         ax[0,i].axhline(k, c="grey", alpha=0.25)

#     vals = df_clusters[f"cluster_{cluster_idx}"].values[::-1]
#     idx = np.argsort(vals)[::-1]
#     vals_prop = (100 * df_clusters[f"cluster_{cluster_idx}"] / df_clusters["total"]).values[::-1]

#     for k in range(3):
#         ax[0,i].add_patch(
#             Rectangle(
#                 (0, idx[k]-0.4),
#                 vals_prop[idx[k]],
#                 0.8,
#                 facecolor="none",
#                 edgecolor="k",
#                 linewidth=2
#             )
#         )
#         ax[0,i].text(
#             vals_prop[idx[k]] / 2 if vals_prop[idx[k]] > 50 else vals_prop[idx[k]] + 15,
#             idx[k],
#             f"{vals[idx[k]]}",
#             horizontalalignment='center',
#             verticalalignment='center',
#             fontsize=12,
#             color="w" if vals_prop[idx[k]] > 50 else "k"
#         )


# for i,cluster_idx in zip(range(4), [1,4,2,3]):
#     ax[1,i].remove()
#     ax_2 = fig.add_subplot(2, 4, 5+i, projection='polar')

#     # If you want the first axis to be on top:
#     # ax.set_projection("polar")
#     theta_offset = 7*pi / 4
#     # theta_offset_
#     # ax_2.set_theta_offset(theta_offset)
#     ax_2.set_theta_direction(-1)

#     # Draw one axe per variable + add labels
#     ax_2.set_xticks((angles[:-1] + theta_offset) % (2 * pi))
#     ax_2.set_xticklabels(labels=categories, fontsize=12)
#     ax_2.tick_params(pad=0.075)


#     # Draw ylabels
#     ax_2.set_rlabel_position(45)
#     ax_2.set_yticks([0.5, 1], ["0.5", "1.0"], color="grey", size=10)
#     # ax_2.set_yticks([0.25,0.5,0.75], ["","",""], color="grey")
#     ax_2.set_ylim(0,1)


#     # ------- PART 2: Add plots

#     # Plot each individual = each line of the data
#     # I don't make a loop, because plotting more than 3 groups makes the chart unreadable

#     # Ind1

#     values_raw = list(
#         df_corr_output.loc[f"cluster_{cluster_idx}", ["Pth", "G0", "AV"]].values
#     )
#     values_raw.reverse()
#     values_raw += [values_raw[0]]
#     values = np.abs(values_raw)

#     ax_2.plot((angles + theta_offset), values, linewidth=1, linestyle='solid', label=f"cluster {cluster_idx}")
#     ax_2.fill((angles + theta_offset), values, alpha=0.1)
#     ax_2.plot(
#         [angle for angle,val in zip((angles + theta_offset),values_raw) if val < 0],
#         np.abs([val for val in values_raw if val < 0]),
#         "o", color='tab:blue'
#     )

# fig.tight_layout()
# plt.subplots_adjust(wspace=0.075, hspace=0.05)

# plt.show()

## Intra and inter cluster properties

In this section, we will investigate the intra and inter cluster properties, that is the difference in correlation between the lines of the same cluster and the lines of two distinct clusters.

In [ ]:
C = np.zeros((n_clusters, n_clusters))

for k1, k2 in itt.product(clusters_indices, clusters_indices):

    C[k1 - 1, k2 - 1] = corr[clusters_indices[k1], :][:, clusters_indices[k2]].mean()

plt.figure(dpi=125)
plt.imshow(
    C, cmap="jet", vmin=0, vmax=1, extent=[0.5, n_clusters + 0.5, 0.5, n_clusters + 0.5]
)
plt.xticks(list(range(1, n_clusters + 1)))
plt.yticks(list(range(1, n_clusters + 1)), labels=list(range(1, n_clusters + 1))[::-1])
plt.colorbar()

plt.xlabel("Cluster")
plt.ylabel("Cluster")
plt.title("Average pearson correlation coefficient between clusters")

C_intra = np.diagonal(C).mean()
C_inter = 0
for k in clusters_indices:
    C_inter += corr[clusters_indices[k], :][
        :, [i for i in range(len(lines_names)) if i not in clusters_indices[k2]]
    ].mean()
C_inter /= n_clusters

print("Average intra cluster correlation:", f"{C_intra:.3f}")
print("Average inter cluster correlation:", f"{C_inter:.3f}")
print("Ratio intra correlation over inter correlation:", f"{C_intra/C_inter:.3f}")

In [ ]:
# # ------- PART 1: Create background

# # number of variable
# categories = [r"$A_V^{tot}$", r"$G_0$", r"$P_{th}$"] #list(df_corr_output)[1:]
# N = len(categories)
# print(N)

# # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
# angles = [n / float(N) * 2 * pi for n in range(N)]
# angles += angles[:1]

# # Initialise the spider plot
# ax = plt.subplot(111, polar=True) #figsize=(8,8),
# ax.set_title("Absolute linear correlations for clusters")

# # If you want the first axis to be on top:
# ax.set_theta_offset(pi / 2)
# ax.set_theta_direction(-1)

# # Draw one axe per variable + add labels
# plt.xticks(angles[:-1], categories)

# # Draw ylabels
# ax.set_rlabel_position(0)
# plt.yticks([0.25,0.5,0.75], ["0.25","0.5","0.75"], color="grey", size=10)
# plt.ylim(0,1)


# # ------- PART 2: Add plots

# # Plot each individual = each line of the data
# # I don't make a loop, because plotting more than 3 groups makes the chart unreadable

# # Ind1
# for cluster_idx in range(1,5):
#     values = list(
#         df_corr_output.loc[f"cluster_{cluster_idx}", ["Pth", "G0", "AV"]].values
#     )
#     values.reverse()
#     values += [values[0]]
#     values = np.abs(values)

#     ax.plot(angles, values, linewidth=1, linestyle='solid', label=f"cluster {cluster_idx}")
#     ax.fill(angles, values, alpha=0.1)
#     ax.plot(angles, values, linewidth=1, linestyle='solid', label=f"cluster {cluster_idx}")

# # # Ind2
# # values=df.loc[1].drop('group').values.flatten().tolist()
# # values += values[:1]
# # ax.plot(angles, values, linewidth=1, linestyle='solid', label="group B")
# # ax.fill(angles, values, 'r', alpha=0.1)

# # Add legend
# plt.legend(loc='upper right')#, bbox_to_anchor=(0.1, 0.1))

# # Show the graph
# plt.show()

## Principal components by cluster

In [ ]:
pca = PCA().fit(X_train)

plt.figure(dpi=125)
plt.semilogy(pca.explained_variance_)
plt.title(f"All lines ({len(lines_names)} lines)")

print(pca.explained_variance_[:1000].sum() / pca.explained_variance_.sum())

In [ ]:
cols = 2
rows = ceil(n_clusters / cols)

plt.figure(figsize=(6.4, 0.5 * cols * 4.8), dpi=125)

for k in clusters_indices:

    pca = PCA().fit(X_train[clusters_indices[k]])

    plt.subplot(rows, cols, k)

    plt.semilogy(pca.explained_variance_)
    plt.xlabel("Principal component")
    plt.ylabel("Explained variance")
    plt.title(f"Cluster n°{k} ({len(clusters_indices[k])} lines)")

plt.tight_layout()
plt.show()

## Advanced analysis

This section proposes to investigate the distribution of a particular molecule. The default example is for $H_2$ but you can easily chose the molecule that you want by modifying the value of `molecule`.

In [ ]:
def search(clusters, molecules):
    if isinstance(clusters, int):
        clusters = [clusters]
    if isinstance(molecules, str):
        molecules = [molecules]

    if clusters is None:
        clusters = list(range(n_clusters))

    selection = []
    for k in clusters:
        selection.extend(clusters_lines[k])
    if molecules is not None:
        selection = filter_molecules(selection, molecules)

    return selection

In [ ]:
molecule = "h2"

print("Distribution of H2 lines")

for k in clusters_lines:
    print(f"\nCluster n°{k}:")
    s = search(k, molecule)
    if len(s) == 0:
        print("No lines")
    elif len(s) == all_molecules_number[all_molecules.index(molecule)]:
        print("All lines")
    elif len(s) > 0.05 * all_molecules_number[all_molecules.index(molecule)]:
        print("Most of the lines")
    else:
        print(*s, sep="  ")